In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data into a Pandas DataFrame
file_path = "merged_julia_final.csv"  # Update this path if necessary
df = pd.read_csv(file_path)

# Split the data into training, test, and evaluation sets (e.g., 70%, 20%, 10%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, eval_df = train_test_split(temp_df, test_size=0.333, random_state=42)  # 0.333 * 30% = ~10%

# Save each subset to a separate CSV file
train_df.to_csv("train_set.csv", index=False)
test_df.to_csv("test_set.csv", index=False)
eval_df.to_csv("eval_set.csv", index=False)

print("Datasets saved as train_set.csv, test_set.csv, and eval_set.csv.")


Datasets saved as train_set.csv, test_set.csv, and eval_set.csv.


In [ ]:
df = df.dropna()
# Split the data into training, test, and evaluation sets (e.g., 70%, 20%, 10%)
train_df_doc, temp_df_doc = train_test_split(df, test_size=0.3, random_state=42)
test_df_doc, eval_df_doc = train_test_split(temp_df_nodoc, test_size=0.333, random_state=42)  # 0.333 * 30% = ~10%

# Save each subset to a separate CSV file
train_df_doc.to_csv("train_set_doc.csv", index=False)
test_df_doc.to_csv("test_set_doc.csv", index=False)
eval_df_doc.to_csv("eval_set_doc.csv", index=False)

print("Datasets saved as train_set_doc.csv, test_set_doc.csv, and eval_set_doc.csv.")


Datasets saved as train_set_doc.csv, test_set_doc.csv, and eval_set_doc.csv.


In [5]:
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")
print(f"Number of rows in evaluation set: {len(eval_df)}")

Number of rows in training set: 697571
Number of rows in test set: 199406
Number of rows in evaluation set: 99554


In [6]:
print(f"Number of rows in training set: {len(train_df_doc)}")
print(f"Number of rows in test set: {len(test_df_doc)}")
print(f"Number of rows in evaluation set: {len(eval_df_doc)}")

Number of rows in training set: 125048
Number of rows in test set: 35745
Number of rows in evaluation set: 17847


In [ ]:
# Combine docstring and function signature as input
train_df['input_text'] = train_df['function_signature']
train_df['target_text'] = train_df['function_body']
        
eval_df['input_text'] = eval_df['function_signature']
eval_df['target_text'] = eval_df['function_body']
        
test_df['input_text'] = test_df['function_signature']
test_df['target_text'] = test_df['function_body']
        
# # Split into training, evaluation, and test sets
trainingSetInputs = train_df['input_text'].tolist()
trainingSetTargets = train_df['target_text'].tolist()
        
evaluationSetInputs = eval_df['input_text'].tolist()
evalutionSetTargets = eval_df['target_text'].tolist()
        
testSetInputs = test_df['input_text'].tolist()
testSetTargets = test_df['target_text'].tolist()

In [ ]:
# Combine docstring and function signature as input
train_df_doc['input_text'] = train_df_doc['docstring'] + "\n" + train_df_doc['function_signature']
train_df_doc['target_text'] = train_df_doc['function_body']
        
eval_df_doc['input_text'] = eval_df_doc['docstring'] + "\n" + eval_df_doc['function_signature']
eval_df_doc['target_text'] = eval_df_doc['function_body']
        
test_df_doc['input_text'] = test_df_doc['docstring'] + "\n" + test_df_doc['function_signature']
test_df_doc['target_text'] = test_df_doc['function_body']
        
# # Split into training, evaluation, and test sets
trainingSetDocInputs = train_df_doc['input_text'].tolist()
trainingSetDocTargets = train_df_doc['target_text'].tolist()
        
evaluationSetDocInputs = eval_df_doc['input_text'].tolist()
evalutionSetDocTargets = eval_df_doc['target_text'].tolist()
        
testSetDocInputs = test_df_doc['input_text'].tolist()
testSetDocTargets = test_df_doc['target_text'].tolist()

In [ ]:
def makeDataset(input_text, target_text, tokenizer, tokens_per_batch=512):
    # Set padding token if not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    max_length = tokens_per_batch  # Define a reasonable max_length for inputs and targets

    # Tokenize the input and target texts
    input_encodings = tokenizer(input_text, padding='max_length', truncation=True, return_tensors='pt', max_length=max_length).to("cuda")
    target_encodings = tokenizer(target_text, padding='max_length', truncation=True, return_tensors='pt', max_length=max_length).to("cuda")
    
    # Ensure labels are aligned with input
    target_ids = target_encodings['input_ids']
    target_ids[target_ids == tokenizer.pad_token_id] = -100  # Ignore padding tokens in the loss computation

    return Dataset.from_dict({
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_ids
    })

In [ ]:
#without docstring
trainingSet = makeDataset(trainingSetInputs, trainingSetTargets, tokenizer)
evaluationSet = makeDataset(evaluationSetInputs, evalutionSetTargets, tokenizer)
testSet = makeDataset(testSetInputs, testSetTargets, tokenizer)

#with docstring
trainingSetDoc = makeDataset(trainingSetDocInputs, trainingSetDocTargets, tokenizer)
evaluationSetDoc = makeDataset(evaluationSetDocInputs, evalutionSetDocTargets, tokenizer)
testSetDoc = makeDataset(testSetDocInputs, testSetDocTargets, tokenizer)


trainingSet.save_to_disk("datasets")
evaluationSet.save_to_disk("datasets")
testSet.save_to_disk("datasets")

trainingSetDoc.save_to_disk("datasets")
evaluationSetDoc.save_to_disk("datasets")
testSetDoc.save_to_disk("datasets")